In [1]:
import os
import numpy as np
import subprocess
from oct2py import octave

In [2]:
path2add = os.path.expanduser("~/work/nbs/mplot")
print(path2add)

if os.path.exists(path2add):
    octave.addpath(path2add)
    print("Path correctly added to octave instance.")
else:
    print("The path is not available")

/home/macu/work/nbs/mplot

Path correctly added to octave instance.


In [3]:
! ls {path2add}

 MASS_s2.m    SPLAT.m	 getPaaFactor.m   piecewiseSplat.m  '~$README.docx'
 README.pdf   SimMat.m	 paa.m		  runsplat.m


In [4]:
subsequence_length = 800

In [5]:
## -- Classes & types
from dataclasses import dataclass, field
from typing import List
import warnings 

In [6]:
@dataclass
class MatlabMatrix: 
    filename : str = ""
    matname : str = ""
    data : List [ float ] = None
    data_dict : dict = field (default_factory = dict)
    smoothing_window_len : int = None
    path : str = None
    
    def load(self, numcol : int, print_flag : bool = False, path = "./"):
        if self.path is None:
            self.path = path

        dir = self.path + '/' + self.filename
        
        self.dict = octave.load(dir)
        data = self.dict[self.matname]
        if print_flag: print(data.dtype)
        self.data = np.array(data[:,numcol])
        return self.data
    def smooth(self, window_len=11, print_flag = False):
        # window_len: smoothing window size needs, which must be odd number,
        # Step 1: Apply the moving average to the main part of the data using convolution
        self.smoothing_window_len = window_len
        if self.smoothing_window_len % 2 == 0:
            warnings.warn("Window len must be odd! Adding 1 to your length.")
            self.smoothing_window_len += 1
        if print_flag: print("---> About to get out0")
        out0 = np.convolve(
            self.data, 
            np.ones(self.smoothing_window_len, dtype=int),
            'valid'
        ) / self.smoothing_window_len
    
        # Step 2: Handle the beginning of the array (start)
        # Use cumulative sum and then average it by the increasing window size
    
        if print_flag : 
            print("out0 ~", out0.shape)
            print("---> About to get start")
    
        r = np.arange(1, self.smoothing_window_len-1, 2)
    
        if print_flag: print("r", r)
    
        start = np.cumsum(self.data[:self.smoothing_window_len-1])[::2] / r
    
        if print_flag:
            print("start", start)
            print("---> About to get stop")
    
        # Step 3: Handle the end of the array (stop)
        # Reverse the array, use cumulative sum, and then average it by the increasing window size
    
        stop = (np.cumsum(self.data[:-self.smoothing_window_len:-1])[::2] / r)[::-1]

        if print_flag: 
            print("stop", stop)
    
        # Step 4: Combine the start, middle, and end parts together
        return np.concatenate((start, out0, stop))



In [7]:
penguins_sample = MatlabMatrix(
    filename = 'MP_first_test_penguin_sample', 
    matname = 'penguin_sample',
    path = os.path.expanduser('~/work/nbs_pipeline/octave-tests')
)
penguins_sample.load( numcol = 0, print_flag = True )




float64


array([0.253906 , 0.259033 , 0.269287 , ..., 0.0559082, 0.064209 ,
       0.059082 ])

In [8]:
len(penguins_sample.data)

109842

In [9]:
%load_ext oct2py.ipython
octave.register_graphics_toolkit('fltk')
octave.graphics_toolkit('fltk')

'qt'

In [10]:
octave.set(0, 'defaultfigurevisible', 'on');

In [25]:
%reload_ext oct2py.ipython
mp, t = octave.runsplat(
    80, 
    penguins_sample.data[:1000], 
    np.nan, 
    1, 
    False, 
    True, 
    False, 
    5000, 
    nout=2
)




opción Splat
1
plotting inside
1
    SPLAT at line 62 column 5
    runsplat at line 48 column 18
    _pyeval at line 57 column 30

plotting antes del if
1
Ploteate
1
--> drawnow
drawnow -->


In [22]:
print("mp ~", mp.shape)
print("time_splat =", t)

mp ~ (921, 921)
time_splat = 6.547044992446899


In [ ]:
print("mp ~", mp.shape)
print("splat time ~", t)

In [ ]:
octave.push('subsequence_length', int(80))
octave.push('timeseriesA', penguins_sample.data[:1000])
octave.push('timeseriesB', np.nan)
octave.push('multiresolution', int(1))
octave.push('calibration', False)
octave.push('display_mplot', True)
octave.push('piecewise', False)
octave.push('patch_size', int(5000))

In [ ]:
#octave.SPLAT(penguins_sample.data[:1000], 80, np.nan, 1, 0, 0)

In [ ]:
octave.eval("runsplat")

In [ ]:
%%octave 
subsequence_length=800;
timeSeriesA = timeSeriesA;
timeSeriesB = nan
multiresolution = 1;
calibration = false
display_mplot = true
picewise = 0;
patch_size = 5000;
runsplat()

In [ ]:
%%octave

SPLAT(timeSeriesA, subsequence_length, timeSeriesB, 1, 0, 0);

In [ ]:
#--> No funciona en este modo
#octave.runsplat()